In [1]:
import os


proxies = {
    'http': 'http://localhost:7890',
    'https': 'http://localhost:7890',
}

os.environ['HTTPS_PROXY'] = proxies['https']


In [1]:
%pip install openai


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/67/78/7588a047e458cb8075a4089d721d7af5e143ff85a2388d4a28c530be0494/openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/e6/02/a2cff6306177ae6bc73bc0665065de51dfb3b9db7373e122e2735faf0d97/tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/c5/22/6bd9f9a90807ef11e4c4a3ec19099e24f2b6f4040ee568f71ed0e5fdb6d1/aiohttp-3.8.4-cp311-cp311-win_amd64.whl (317 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f0/eb/fcb708c7bf5056045e9e98f62b93bd7467eb718b0202e7698eb11d66416c/attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/3d/22/35539dddb1971eb8dc88bb19d22d636eb9efe1ad7549d2f319a7951cbbe7/multidict-6.0.4-cp311-cp311-win_amd64.whl (28 kB)
  Using cached https://


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 微调

- <https://platform.openai.com/docs/guides/fine-tuning>
- <https://platform.openai.com/docs/api-reference/fine-tunes>

In [8]:
import openai


openai.Model.retrieve(id="text-davinci-003")


<Model model id=text-davinci-003 at 0x1f8995563f0> JSON: {
  "id": "text-davinci-003",
  "object": "model",
  "created": 1669599635,
  "owned_by": "openai-internal",
  "permission": [
    {
      "id": "modelperm-jepinXYt59ncUQrjQEIUEDyC",
      "object": "model_permission",
      "created": 1688551385,
      "allow_create_engine": false,
      "allow_sampling": true,
      "allow_logprobs": true,
      "allow_search_indices": false,
      "allow_view": true,
      "allow_fine_tuning": false,
      "organization": "*",
      "group": null,
      "is_blocking": false
    }
  ],
  "root": "text-davinci-003",
  "parent": null
}

In [9]:
# 多模型


tuning_model = {
    'openai': {
        'input_name': 'prompt',
        'output_name': 'completion',
        'input_format': lambda x: f'{x} ->',
        'output_format': lambda x: f' {x}',
    },
    'chatglm': {
        'input_name': 'content',
        'output_name': 'summary',
        'input_format': lambda x: x,
        'output_format': lambda x: x,
    },
}


In [12]:
# 标注数据结构
# 采用下述链接中的清华数据作为测试训练数据集
# https://github.com/THUDM/ChatGLM2-6B/blob/main/ptuning/README.md
import json


# ($prompt, $completion)
# ($content, $summary)
tuning_data = []

# 这里实际是自己手动构造的文件
with open(r'data/tuning/三月七.jl', 'r', encoding='utf-8') as fr:
    for item in fr.readlines():
        item = json.loads(item)
        tuning_data.append((item['content'].strip(), item['summary'].strip()))


tuning_data


[('三月七，关于自己·名字', '名字是我自己取的，大家也叫我三月、小三月…你呢?你想叫我什么?'),
 ('三月七，关于自己·过去', '我的过去，或许不在从前，而是在我的未来里。所以我一定会一站站走下去，哪怕有一天…没有列车。'),
 ('三月七，闲谈·果汁', '果汁…一说就觉得馋，一馋就忒想喝，一喝就…停不下来…'),
 ('三月七，闲谈·照片', '照片当然不是现实，但如果有足够多的照片，是不是就能更接近现实一些呢?'),
 ('三月七，爱好', '拍照，写日记，还用说嘛~'),
 ('三月七，烦恼', '要是没人和我聊天，我就闷得要命。但在列车上的时候，姬子很忙，杨叔很忙，帕姆很忙，丹恒倒是不忙，可他不喜欢聊天……')]

In [13]:
import json


# 生成微调数据
tm = tuning_model['openai']
data = [{tm['input_name']: tm['input_format'](
    item[0]), tm['output_name']: tm['output_format'](item[1])} for item in tuning_data]

# 生成微调数据文件
tuning_data_file = r'/Temp/out/data.jl'
with open(tuning_data_file, 'w', encoding='utf-8') as fw:
    fw.writelines(
        [json.dumps(item, ensure_ascii=False) + '\n' for item in data])


In [14]:
import openai


# 上传微调数据文件
upload_response = openai.File.create(
    file=open(tuning_data_file, "rb"),
    purpose='fine-tune'
)

upload_response


<File file id=file-T7lJiIqSfQ52zW5XvfZRIEFA at 0x1f899556b10> JSON: {
  "object": "file",
  "id": "file-T7lJiIqSfQ52zW5XvfZRIEFA",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 984,
  "created_at": 1689184051,
  "status": "uploaded",
  "status_details": null
}

In [15]:
# 执行微调请求
tuning_response = openai.FineTune.create(training_file=upload_response['id'])
tuning_response


<FineTune fine-tune id=ft-doQAQZ1i6qdy87QJC5tVsYw5 at 0x1f899556810> JSON: {
  "object": "fine-tune",
  "id": "ft-doQAQZ1i6qdy87QJC5tVsYw5",
  "hyperparams": {
    "n_epochs": 4,
    "batch_size": null,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": null
  },
  "organization_id": "org-XTGud4J4OK1d770iUX8UDn92",
  "model": "curie",
  "training_files": [
    {
      "object": "file",
      "id": "file-T7lJiIqSfQ52zW5XvfZRIEFA",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 984,
      "created_at": 1689184051,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [],
  "result_files": [],
  "created_at": 1689184065,
  "updated_at": 1689184065,
  "status": "pending",
  "fine_tuned_model": null,
  "events": [
    {
      "object": "fine-tune-event",
      "level": "info",
      "message": "Created fine-tune: ft-doQAQZ1i6qdy87QJC5tVsYw5",
      "created_at": 1689184065
    }
  ]
}

In [18]:
import time


# 查看微调状态
job_id = tuning_response['id']

status = openai.FineTune.retrieve(id=job_id)["status"]
if status not in ["succeeded", "failed"]:
    print(f'Job not in terminal status: {status}. Waiting.')
    while status not in ["succeeded", "failed"]:
        time.sleep(30)
        status = openai.FineTune.retrieve(id=job_id)["status"]
        print(f'Status: {status}')
else:
    print(f'Finetune job {job_id} finished with status: {status}')


openai.FineTune.list()


Job not in terminal status: pending. Waiting.
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pending
Status: pe

<OpenAIObject list at 0x1f8996c0b90> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine-tune",
      "id": "ft-xhs5qPiwIzGq8xJwcZk1ZAUZ",
      "hyperparams": {
        "n_epochs": 4,
        "batch_size": null,
        "prompt_loss_weight": 0.01,
        "learning_rate_multiplier": null
      },
      "organization_id": "org-XTGud4J4OK1d770iUX8UDn92",
      "model": "davinci",
      "training_files": [
        {
          "object": "file",
          "id": "file-PFzuCvPcEwMguzKafjhkKr96",
          "purpose": "fine-tune",
          "filename": "C:\\Users\\admin\\Desktop\\data_prepared.jsonl",
          "bytes": 2353676,
          "created_at": 1682046438,
          "status": "processed",
          "status_details": null
        }
      ],
      "validation_files": [],
      "result_files": [],
      "created_at": 1682046439,
      "updated_at": 1682046554,
      "status": "failed",
      "fine_tuned_model": null
    },
    {
      "object": "fine-tune",
      "id": "ft

In [19]:
# 查看微调结果
fine_tune_response = openai.FineTune.retrieve(id=job_id)
new_model_name = fine_tune_response['fine_tuned_model']
openai.Model.retrieve(id=new_model_name)


<Model model id=curie:ft-personal-2023-07-12-19-59-40 at 0x1f89966d670> JSON: {
  "id": "curie:ft-personal-2023-07-12-19-59-40",
  "object": "model",
  "created": 1689191980,
  "owned_by": "user-bwjzwcjiml01n6knupzblv8t",
  "permission": [
    {
      "id": "snapperm-ZxQcoPaFK63Cc83MSsils2fs",
      "object": "model_permission",
      "created": 1689191980,
      "allow_create_engine": true,
      "allow_sampling": true,
      "allow_logprobs": true,
      "allow_search_indices": false,
      "allow_view": true,
      "allow_fine_tuning": true,
      "organization": "org-XTGud4J4OK1d770iUX8UDn92",
      "group": null,
      "is_blocking": false
    }
  ],
  "root": "curie:2020-05-03",
  "parent": "curie:2020-05-03"
}

In [20]:
response = openai.Completion.create(
    model=new_model_name,
    prompt="三月七是个怎样的姑娘？",
    max_tokens=128,
    temperature=0.3
)

response


<OpenAIObject text_completion id=cmpl-7baI07on7y57NLlgXGLmhJQzukIRD at 0x1f8996f5490> JSON: {
  "id": "cmpl-7baI07on7y57NLlgXGLmhJQzukIRD",
  "object": "text_completion",
  "created": 1689191988,
  "model": "curie:ft-personal-2023-07-12-19-59-40",
  "choices": [
    {
      "text": "\u201d\n\n\u201c\u6211\u4e0d\u77e5\u9053\uff0c\u6211\u53ea\u77e5\u9053\u5979\u662f\u4e00\u4e2a\u600e\u6837\u7684\u59d1\u5a18\uff0c\u4e00\u4e2a\u600e\u6837\u7684\u59d1\u5a18\uff0c\u4e00\u4e2a\u600e\u6837\u7684\u59d1\u5a18\uff0c\u4e00\u4e2a\u600e\u6837\u7684\u59d1\u5a18\uff01\u201d\n\n\u201c\u4f60\u77e5\u9053\u5979\u662f\u4e00\u4e2a\u600e\u6837\u7684\u59d1\u5a18\uff1f\u201d\n",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 21,
    "completion_tokens": 128,
    "total_tokens": 149
  }
}

In [21]:
response['choices'][0]['text']


'”\n\n“我不知道，我只知道她是一个怎样的姑娘，一个怎样的姑娘，一个怎样的姑娘，一个怎样的姑娘！”\n\n“你知道她是一个怎样的姑娘？”\n'